# IBM Applied Data Science Capstone Course by Coursera
# Week 5 Final Report

## BEST LOCATION TO BUILD A FEDERAL LEARNING INSTITUTION IN SOUTHERN EDO STATE NIGERIA 

### Build a dataframe of neighborhoods in SOUTHERN EDO STATE NIGERIA  by web scraping the data from Wikipedia page
### Get the geographical coordinates of the neighborhoods
### Obtain the venue data for the neighborhoods from Foursquare API
### Explore and cluster the neighborhoods
### Select the best cluster to build a federal learning institution

In [16]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [8]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Populated_places_in_Edo_State").text
data

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Category:Populated places in Edo State - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"d916f5af-95fa-4be2-b95c-9c431ce38d0e","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Populated_places_in_Edo_State","wgTitle":"Populated places in Edo State","wgCurRevisionId":585634912,"wgRevisionId":585634912,"wgArticleId":27666217,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Populated places in Nigeria by state","Edo State"],"wgPageContentLanguage":"en","wgPa

In [9]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [10]:
# create a list to store neighborhood data
neighborhoodList = []

In [11]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [12]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Agenebode
1,Auchi
2,Benin City
3,Egor
4,Ekpoma


In [13]:
# print the number of rows of the dataframe
kl_df.shape

(35, 1)

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [17]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [18]:
coords

[[3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.1549000000000547, 101.69638000000003],
 [3.147890000000075, 101.69405000000006],
 [3.1586300000000733, 101.70930000000004],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.6940500

In [19]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [20]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [21]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(35, 3)


,Neighborhood,Latitude,Longitude
0,Agenebode,3.14789,101.69405
1,Auchi,3.14789,101.69405
2,Benin City,3.14789,101.69405
3,Egor,3.14789,101.69405
4,Ekpoma,3.14789,101.69405
5,Emokweme,3.14789,101.69405
6,Esan Central,3.15490,101.69638
7,Esan North-East,3.14789,101.69405
8,Esan West,3.15863,101.70930
9,Eso (town),3.14789,101.69405


In [22]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

 # Create a map of Edo state with neighborhoods superimposed on top

In [29]:
# get the coordinates of edo state
address = 'Edo State, Nigeria'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Edo State, Nigeria {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Edo State, Nigeria 6.6076575, 5.9722713.


In [30]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [31]:
# save the map as HTML file
map_kl.save('map_kl.html')